In [4]:
import pandas as pd
import numpy as np
from pprint import pprint
import pickle
import glob, os
import nltk
from nltk.util import ngrams
from nltk import bigrams, trigrams, word_tokenize
from collections import Counter, defaultdict
import random
os.chdir("../quoteAIsData")

Using 4 gram N-Gram model

In [5]:
names = ["DAVID", "DIEGO", "ERIC", "GWYN", "JETT", "MARS", "MILES", "MILO", "PARSA"]
def safe_insert_list(dictionary, key, value):
    if key in dictionary:
        dictionary[key].append(value)
    else:
        dictionary[key] = list()
        dictionary[key].append(value)

def safe_insert_dict(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = {}
    if value not in dictionary[key]:
        dictionary[key][value] = 0

def line_counts(dictionary):
    for key in dictionary:
        print(key, len(dictionary[key]))
        
def getfileOwner(file):
    name = file.split('.')[0]
    for i in range(0, len(name)):
        if name[i].isdigit():
            name = name [:i]
            break
    return name.upper()


def print_quotegen(quote_gen):
    for k,v  in quote_gen.items():
        print(k)
        print('\t',[ (k2,v2) for k2,v2  in v.items()])
        
def load_quotegen(person):
    return pickle.load(open(person + '.pkl', 'rb') )


In [6]:
quote_dict = {}
for file in glob.glob("*.txt"):
    #not a base file
    if "_" in file:
        continue
    owner = getfileOwner(file)
    f = open(file, 'r', encoding="utf-8")
    lines = f.read()
    lines = lines.split("\n")
    for l in lines:
        if l == "":
            continue
        safe_insert_list(quote_dict, owner, l)
    f.close()
for k in quote_dict:
    print(k, len(quote_dict[k]))

DAVID 2446
DIEGO 3260
ERIC 32224
GWYN 21457
JETT 5554
MARS 23434
MILES 15062
MILO 34714
PARSA 9714


In [7]:
def make_quotegen(person='PARSA'):   
    quote_gen = {}

    for sentence in quote_dict[person]:
        token = nltk.word_tokenize(sentence)
        for w1, w2, w3, w4 in ngrams(token, 4, pad_left = True, pad_right=True):
            safe_insert_dict(quote_gen, (w1, w2, w3), w4)
            quote_gen[(w1, w2, w3)][w4] += 1
    
    for w1_w2_w3 in quote_gen:
        total_count = float(sum(quote_gen[w1_w2_w3].values()))
        for w4 in quote_gen[w1_w2_w3]:
            quote_gen[w1_w2_w3][w4] /= total_count

    return quote_gen

for person in quote_dict:
    q = make_quotegen(person)
    pickle.dump(q, open(person + '.pkl', 'wb') )
    print("Done with quotegen " + person)

Done with quotegen DAVID
Done with quotegen DIEGO
Done with quotegen ERIC
Done with quotegen GWYN
Done with quotegen JETT
Done with quotegen MARS
Done with quotegen MILES
Done with quotegen MILO
Done with quotegen PARSA


In [8]:
def generate_sentence(person):
    quote_gen = load_quotegen(person)
    text = [None,None,None]
    sentence_finished = False
    while not sentence_finished:
        r = random.random()
        accumulator = .0

        for word in quote_gen[tuple(text[-3:])].keys():
            accumulator += quote_gen[tuple(text[-3:])][word]

            if accumulator >= r:
                text.append(word)
                break

        if text[-3:] == [None,None,None]:
            sentence_finished = True

    return ' '.join([t for t in text if t])

In [16]:
for i in range(20):
    print("Parsa: " + generate_sentence('PARSA'))

Parsa: yeah but playing league at all means youre buried under 10 miles of overwatch shit
Parsa: and ill be honest , I thought she was a support
Parsa: No I ’ m playing smash at milos house
Parsa: how vile
Parsa: yeah
Parsa: youre playing with a bard , monk , and another fighter
Parsa: It ’ s like setting the pendulum
Parsa: friday
Parsa: it does n't reflect on how you lead
Parsa: or must he remain destitute and teamless
Parsa: Miles collectors cache particles
Parsa: Concede
Parsa: I ’ m all in for extermination
Parsa: there is no skill in dota . and the game has a high skill barrier to entry ?
Parsa: and izoldi is actually trapped in the gem in the sword 's crossguard or pommel
Parsa: EASY
Parsa: i do n't care enough to send in a ballot
Parsa: give me back my Beijing air
Parsa: fucking cancer hero
Parsa: the whole point is to make the 4 sets of gear i requested


In [ ]:
for n in names:
    for i in range(20):
        print(n + ": " + generate_sentence(n))
    print("---------------------------------------")

In [19]:
pickle.dump(quote_dict, open('quotes.pkl', 'wb'))